 Answering Business Questions Using SQL

1. OVERVIEW OF THE DATA

In this project we are going to answer business questions related to Chinook Record Store using SQL. Chinook database is provided as a SQLite database file. Down below we impliment the code which connects the Jupyter Notebook to the Chinook databse.

In [10]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [11]:
#We first need to familiarize with the tables and views in the database

In [12]:
%%sql
SELECT 
      name,
      type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


2. SELECTING ALBUMS TO PURCHASE

The Chinook record store has just signed a deal with a new record label, and we have been tasked with selecting the first three albums that will be added to the store, from a list of four.
All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:


Artist Name	              Genre
Regal	                  Hip-Hop
Red Tone	              Punk
Meteor and the Girls	  Pop
Slim Jim Bites	          Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA

we need to write a query to find out which genres sell the most tracks in the USA. We will write up a summary of the findings, and make a recommendation for the three artists whose albums we should purchase for the store.

In [13]:
%%sql

     WITH t1 AS (SELECT iv.billing_country,
                           il.quantity, 
                           gr.name as genre_name
                      FROM invoice iv
                      JOIN invoice_line il
                        ON iv.invoice_id = il.invoice_id
                      JOIN track tr
                        ON tr.track_id = il.track_id
                      JOIN genre gr
                        ON gr.genre_id = tr.genre_id
                     WHERE iv.billing_country = 'USA'
                   ),

            t2 AS (SELECT genre_name,
                            COUNT(*) AS num_purchases
                       FROM t1
                      GROUP BY 1
                      ORDER BY 2 DESC
                    ) 

        
        SELECT *, 
                ROUND(num_purchases * 100 / 
                      CAST(
                           (SELECT SUM(num_purchases) FROM t2) 
                            AS Float
                          ), 1
                     ) AS percentage_sold
          FROM t2;

             
      
      
      

Done.


genre_name,num_purchases,percentage_sold
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Latin,22,2.1
Pop,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


From the above list it is clear that Rock sells best in USA, i.e. 53.38% (more than half). But we do not have it in the record label.

Below we list all the four genres in the record label according to their sales percentage (highest to lowest) in USA.

Highest of these is Alternate & Punk. It has a sales of 12.37%. And Punk is in the record label.

Next highest is Blues with the sales of 3.43%.
Third highest is Pop with the sales of 2.09%.
Last is Hip-Hop/Rap with only 1.9% sales.
Based on our observation of the sales percentage, we recommend these three artists:-

1.Red Tone

2.Slim Jim Bites

3.Meteor and the Girls

3. Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

We might like to consider whether any extra columns from the employee table explain any variance you see, or whether the variance might instead be indicative of employee performance.

In [14]:
%%sql
SELECT e.first_name || " " || e.last_name Employee_Name,
       e.title Title,
       e.hire_date Hire_Date,
       e.birthdate Birthdate,
       max(invoice_date) latest_invoice_date,
       ROUND(SUM(i.total), 2) Total_Sales_$,
       ROUND(SUM(i.total) * 100 /(SELECT SUM(total)
                                    FROM invoice), 2) Sales_Perc
       
  FROM employee e

LEFT JOIN customer c ON c.support_rep_id = e.employee_id
LEFT JOIN invoice i ON i.customer_id = c.customer_id

WHERE Title = "Sales Support Agent"
GROUP BY 1
ORDER BY 6 DESC;


Done.


Employee_Name,Title,Hire_Date,Birthdate,latest_invoice_date,Total_Sales_$,Sales_Perc
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,1973-08-29 00:00:00,2020-12-30 00:00:00,1731.51,36.77
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,1947-09-19 00:00:00,2020-12-29 00:00:00,1584.0,33.63
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,1965-03-03 00:00:00,2020-12-20 00:00:00,1393.92,29.6


From the above table we see that Jane Peacock holds the highest sales amount of ~ $1731, followed by Margaret Park with ~ $1584. Last is Steve Johnson with ~ $1394 sales amount.

If we look at their hire_date, Jane Peacock joined first i.e. on 4th April 2017. Margaret Park joined in a month's time i.e. on 3rd May 2017. 

Where as Steve Johnson joined a few month's later, i.e. on 17th October 2017. If we look at the latest invoice date of all the three, there is not much difference.

We observe that both Jane Peacock and Steve Johnson hold better sales value compared to Margaret Park, evethough Jane Peacock joined a month earlier and Steve Johnson joined five months later than her. 

If we Look at the birthdate, both Jane Peacock and Steve Johnson are younger to Margaret Park and Jane Peacock is younger to Steve Johnson. 

From this we perceive that the age can also have an effect on the performance of the employees. i.e. younger employees perform better than older employees.

4. Analyzing Sales by Country

Our next task is to analyze the sales data for customers from each different country. 

In particular, We have been directed to calculate data, for each country, on the:

Because there are a number of countries with only one customer, We should group these customers as "Other" in your analysis. 



We are going to write a query that collates data on purchases from different countries.
Where a country has only one customer, collect them into an "Other" group.

The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

For each country, include:
total number of customers
total value of sales
average value of sales per customer
average order value

In [15]:
%%sql
WITH num_of_cstmrs AS
(SELECT country Country,
       count(first_name) No_of_customers       
  FROM customer 
GROUP BY 1
ORDER BY 2 DESC)  

SELECT * FROM num_of_cstmrs;

Done.


Country,No_of_customers
USA,13
Canada,8
Brazil,5
France,5
Germany,4
United Kingdom,3
Czech Republic,2
India,2
Portugal,2
Argentina,1


First we will calculate the total value of sales, average value of sales per customer and average order value for all countries. 

Then we can seperate the countries with only one customer, aggregate their values and rename them as Others. Later we can club the countries with more than one customer and countries with one customer into one table

In [ ]:
%%sql
WITH tot_sales_val AS
(SELECT c.country Country,
       ROUND(SUM(i.total), 2) Tot_Value_Sales,
       ROUND(AVG(i.total), 2) Avg_Order_Value,
       ROUND(SUM(i.total)/(SELECT COUNT(DISTINCT(customer_id)) FROM invoice), 2) Avg_Sales_Value_per_Customer
 
 FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1
ORDER BY 2 DESC),

sales_data_1 AS
(SELECT noc.Country,
       noc.No_of_customers,
       tsv.Tot_Value_Sales,
       tsv.Avg_Sales_Value_per_Customer,
       tsv.Avg_Order_Value
  FROM Num_of_cstmrs noc  
INNER JOIN tot_sales_val tsv ON tsv.Country=noc.Country
 WHERE noc.No_of_customers > 1),

sales_data_2 AS
(SELECT CASE
       WHEN noc.No_of_customers=1 THEN "Others"
       END AS
       Country,
       noc.No_of_customers,
       ROUND(AVG(tsv.Tot_Value_Sales),2) Tot_Value_Sales,
       ROUND(AVG(tsv.Avg_Sales_Value_per_Customer),2) AVG_Sales_Value_per_Customer,
       ROUND(AVG(tsv.Avg_Order_Value),2) AVG_Order_Value
  FROM Num_of_customers noc  
INNER JOIN tot_sales_val tsv ON tsv.Country=noc.Country
 WHERE noc.No_of_customers = 1),

tot_sales_data AS
(SELECT * 
FROM sales_data_1
UNION
SELECT * 
FROM sales_data_2)

SELECT *
  FROM tot_sales_data
ORDER BY 3 DESC;

Above table is in the descending order of total value of sales. And Others (countries with only one customer) is at the bottom of the table.

Here we observe that USA is on top of the list with 13 customers and with ~ $1040 total value of sales. Next is CANADA with 8 customers and ~ $535 total value of sales.

5. Albums vs Individual Tracks

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

purchase a whole album
purchase a collection of one or more individual tracks.
The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). 

When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. 

The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when you are performing an analysis to have 'edge cases' which prevent us from getting a 100% accurate answer to our question. In this instance, we have two edge cases to consider:

Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.

In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks

We will Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:

1. Number of invoices
2. Percentage of invoices

In [ ]:
%%sql
SELECT MAX(invoice_id) total_invoice
  FROM invoice_line;

From the above table we observe that there are total of 614 invoices. Further we will compare the tracks from invoice with that of albums.

In [17]:
%%sql
WITH com_pur AS
(SELECT il.invoice_id invoice_id, il.track_id Pur_tracks, t.album_id album_id
  FROM track t
INNER JOIN invoice_line il ON il.track_id = t.track_id
ORDER BY 1)

SELECT *
  FROM com_pur;

Done.


invoice_id,Pur_tracks,album_id
1,1158,91
1,1159,91
1,1160,91
1,1161,91
1,1162,91
1,1163,91
1,1164,91
1,1165,91
1,1166,91
1,1167,91


From the above table we observe that if the tracks belonging to each distinct invoice, belongs to the same album it is an Album Purchase. If these tracks belong to different albums it is Non Album Purchase.

Below we categorize these invoices into Album and Non Album Purchase. We do this by counting the number of distinct albums belonging to each invoice. Then we compare the Distinct Albums and Number of Albums belonging to each invoice

In [18]:
%%sql
WITH com_pur AS
(SELECT il.invoice_id invoice_id, il.track_id Pur_tracks, t.album_id album_id
  FROM track t
INNER JOIN invoice_line il ON il.track_id = t.track_id
ORDER BY 1)

SELECT invoice_id, COUNT(DISTINCT(album_id)) Distinct_album_id, 
       COUNT(album_id) No_of_album_id,
       CASE
       WHEN COUNT(album_id) - COUNT(DISTINCT(album_id)) <= 2 THEN "Non Album Purchase"
       ELSE "Album Purchase"
       END AS Category 
  FROM com_pur
GROUP BY 1;

Done.


invoice_id,Distinct_album_id,No_of_album_id,Category
1,1,16,Album Purchase
2,10,10,Non Album Purchase
3,2,2,Non Album Purchase
4,8,8,Non Album Purchase
5,1,17,Album Purchase
6,2,2,Non Album Purchase
7,11,11,Non Album Purchase
8,10,10,Non Album Purchase
9,9,9,Non Album Purchase
10,2,2,Non Album Purchase


In [19]:
%%sql
WITH com_pur AS
(SELECT il.invoice_id invoice_id, il.track_id Pur_tracks, t.album_id album_id
  FROM track t
INNER JOIN invoice_line il ON il.track_id = t.track_id
ORDER BY 1),

ctgr_pur AS
(SELECT invoice_id, COUNT(DISTINCT(album_id)) Distinct_album_id, 
       COUNT(album_id) No_of_album_id,
       CASE
       WHEN COUNT(album_id) - COUNT(DISTINCT(album_id)) <= 2 THEN "Non Album Purchase"
       ELSE "Album Purchase"
       END AS Category 
  FROM com_pur
GROUP BY 1)

SELECT (SELECT MAX(invoice_id)
          FROM invoice_line) TOTAL_invoice_id,
       COUNT(invoice_id) No_of_invoice_id,
       (SELECT ROUND(CAST(COUNT(invoice_id) AS FLOAT) * 100 / (SELECT MAX(invoice_id)
                                            FROM invoice_line), 2)) Perc_invoice
  FROM ctgr_pur
WHERE Category = "Non Album Purchase";

Done.


TOTAL_invoice_id,No_of_invoice_id,Perc_invoice
614,480,78.18


From the above table we deduce that out of 614 invoices, 480 invoices are Non Album Purchases. i.e. ~ 78% of purchases are Non Album Purchases.

Conclusions
In this project we answered business queries related to Chinook Record Store using SQL Database. Our analysis led to the following observations:-

We found which genres sell best in the USA. Based on this we listed three albums to be added to the Chinook store. These are:-
Red Tone
Slim Jim Bites
Meteor and the Girls

We analyzed the performance of Sales Support Agents. And found that Jane Peacock and Steve Johnson are doing better than Margaret Parker. We also observed that the age impacts the performance as Jane Peacock is youngest of all and Steve Johnson is younger to Margaret Parker.
We analyzed the sales data for customers from each country. We calculated total number of customers, total value of sales, average value of sales per customer and average order value for each country. We observed that USA is on top of the list with 13 customers and ~ $1040 total value of sales. Next is CANADA with 8 customers and with ~ $535 value of sales.

We calculated the percentage of individual tracks and the percentage of whole album purchased by customers. Our analysis led to the conclusion that the individual tracks purchases (~ 78%) are higher compared to album purchases (~ 22%). This can help the company in deciding whether to buy popular tracks from albums or to buy the whole album from record companies.


